<a href="https://colab.research.google.com/github/Franklyn-SWE/Body-Adiposity-Index-Calculator/blob/main/Copy_of_coercion_model_using_BERT_version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertForSequenceClassification



In [ ]:
#Importing librabries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
#from transformers import DistilBERTTokenizerFast
from transformers import DistilBertTokenizer
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier


from sklearn.preprocessing import MultiLabelBinarizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Importing the first dataset
from pandas import json_normalize
import json

# Loading JSON data
with open('/content/enrichments.articles.json') as f:
    data = json.load(f)

# Flatten the JSON data
enrich_df = json_normalize(data)
print(enrich_df.head(2))  # Display the first 2 rows

                                                 url  isWayBack  \
0  https://www.watfordobserver.co.uk/news/nationa...      False   
1  https://omaha.com/news/state-regional/crime-co...      False   

                                               title  \
0  Gang of five convicted of human trafficking an...   
1  Nebraska officials again sound alarm on human ...   

                                             article  isValid  \
0  At the High Court in Glasgow on Thursday, Mari...     True   
1  LINCOLN — State and federal law enforcement of...     True   

                               article_hash  \
0  a5432c46ff2d91b2d63ccab16be6e1bfcd53df88   
1  815229d337dd3481cbe7d29aff71b0743668f3c1   

                                             summary publish_date  crawl_date  \
0  Detectives said the gang, originally from Roma...   2025-01-09  2025-01-10   
1  LINCOLN — State and federal law enforcement of...          NaN  2025-01-10   

  gdeltx  ...   tah_ner.Location_Source tah_ner

In [ ]:
# Importing the second dataset
from pandas import json_normalize
import json

# Load JSON data
with open('/content/enrichments.articles-27K.json') as f:
    data = json.load(f)

# Flatten the JSON data
enrich27k_df = json_normalize(data)
print(enrich27k_df.head(2))  # Display the first 2 rows

                                                 url  isWayBack  \
0  https://wibc.com/431751/man-pleads-guilty-afte...      False   
1  https://nashvilletnnewssource.com/man-charged-...      False   

                                               title  \
0  Man Pleads Guilty After Multiple Charges, Incl...   
1  Man charged with 68 counts of distribution of ...   

                                             article  isValid  \
0  WESTFIELD — A man who authorities claim used S...     True   
1  Reading Time: < 1 minute\n\n\n\n\n\n\n\nHOPKIN...     True   

                               article_hash  \
0  e23431e095c00360c6040b44813f3a94120fbc73   
1  fcc667b56e8e58016ba8aaf27f73580d336d2f46   

                                             summary publish_date  \
0  WESTFIELD — A man who authorities claim used S...   2024-08-30   
1  The Hopkinsville Police Department said detect...   2024-08-31   

             authors  crawl_date  ... tah_ner.Trafficker_Age_Group  \
0    [Jake Mc

In [ ]:
 # Checking the total sum of the missing values in coercion keys
print(enrich27k_df['coercion.keys'].isnull().sum())

523


In [ ]:
# Drop rows with NaN in 'coercion.keys' in place
enrich27k_df.dropna(subset=['coercion.keys'], inplace=True)

In [ ]:
print(enrich27k_df['coercion.keys'].isnull().sum())

0


In [ ]:
# Extracting text data
new_df2 = enrich27k_df[['article','title','summary']].copy()
new_df2.head(2)

,article,title,summary
0,WESTFIELD — A man who authorities claim used S...,"Man Pleads Guilty After Multiple Charges, Incl...",WESTFIELD — A man who authorities claim used S...
1,Reading Time: < 1 minute\n\n\n\n\n\n\n\nHOPKIN...,Man charged with 68 counts of distribution of ...,The Hopkinsville Police Department said detect...


In [ ]:
# combining text data of the second dataset
new_df2['combined_text_df'] = enrich27k_df['article'] + " " + enrich27k_df['title'] + " " + enrich27k_df['summary']

In [ ]:
# Viewing the combined text data (dataset 2)
new_df2['combined_text_df']

,combined_text_df
0,WESTFIELD — A man who authorities claim used S...
1,Reading Time: < 1 minute\n\n\n\n\n\n\n\nHOPKIN...
2,"In a statement, Paris prosecutor Laure Beccuau..."
3,A woman allegedly killed her boyfriend's 9-yea...
4,Son Heung-min's father summarily indicted on c...
...,...
27356,The National Redress Scheme provides support t...
27357,"Dariel Baques was arrested on Friday, left, an..."
27358,"Dariel Baques was arrested on Friday, left, an..."
27359,A Norwegian man who was found in possession of...


In [ ]:
# enrich2k target feature of dataset 2 (coercion keys)
enrich27k_df['coercion.keys'].head(2)

,coercion.keys
0,"[Minor, Sexual Abuse]"
1,[Minor]


In [ ]:
new_df = enrich_df[['article','title','summary','coercion.keys']].copy()
new_df.head(1)

,article,title,summary,coercion.keys
0,"At the High Court in Glasgow on Thursday, Mari...",Gang of five convicted of human trafficking an...,"Detectives said the gang, originally from Roma...",NaN


In [ ]:
# Combining the text data to create a dataframe (dataset 1)
new_df['combined_text_df1'] = enrich_df['article'] + " " + enrich_df['title'] + " " + enrich_df['summary']
new_df['combined_text_df1']

,combined_text_df1
0,"At the High Court in Glasgow on Thursday, Mari..."
1,LINCOLN — State and federal law enforcement of...
2,"At the High Court in Glasgow on Thursday, Mari..."
3,"At the High Court in Glasgow on Thursday, Mari..."
4,"If you didn't know, January is National Human ..."
...,...
2979,State Police investigators out of Lyons report...
2980,Attorney General Alan Wilson | Attorney Genera...
2981,Attorney General Alan Wilson | Attorney Genera...
2982,"MICHIGAN CITY, Ind. (WNDU) - A 29-year-old Mic..."


In [ ]:
# Cleaning dataset 1
new_df['combined_text_df1'].isnull().sum()

np.int64(0)

In [ ]:
# Checking the target feature for null values
new_df['coercion.keys'].isnull().sum()

np.int64(62)

In [ ]:
# Drop rows with NaN in 'coercion.keys' in place
new_df.dropna(subset=['coercion.keys'], inplace=True)

# Checking the target feature for null values
new_df['coercion.keys'].isnull().sum()

np.int64(0)

In [ ]:
# Cocatenate both Text DF (dataset 1 and 2 text features)
combined_text_df = pd.concat([
    new_df[['combined_text_df1']].rename(columns={'combined_text_df1': 'combined_text'}),
    new_df2[['combined_text_df']].rename(columns={'combined_text_df': 'combined_text'})
], axis=0, ignore_index=True)


In [ ]:
# Concatenate both target feature columns into a single Series
combined_coercion_keys = pd.concat([new_df['coercion.keys'], enrich27k_df['coercion.keys']], ignore_index=True)

# Display the shape of the combined target feature
print(combined_coercion_keys.shape)


(29760,)


In [ ]:
print(len(combined_coercion_keys))  # Should be 29760
print(len(combined_text_df))  # Should be 29760

29760
29760


In [ ]:
X = combined_text_df['combined_text'].values  # This gives us the text as a 1D numpy array
print("X shape before reshaping:", X.shape)  # It should be (29760,)

X shape before reshaping: (29760,)


In [ ]:
# Reshape it to be 2D (29760, 1)
X = X.reshape(-1, 1)  #  X has shape (29760, 1)
print("X shape after reshaping:", X.shape)

X shape after reshaping: (29760, 1)


In [ ]:
# asigning the feature target to y
y = combined_coercion_keys
print(y)

0              [Forced, Minor]
1                      [Minor]
2                      [Minor]
3                      [Minor]
4              [Forced, Minor]
                 ...          
29755           [Sexual Abuse]
29756    [Minor, Sexual Abuse]
29757                  [Minor]
29758    [Minor, Sexual Abuse]
29759                  [Minor]
Name: coercion.keys, Length: 29760, dtype: object


In [ ]:
print(y.value_counts())  # Check class distribution

coercion.keys
[Minor]                                                                                                                 10432
[Minor, Sexual Abuse]                                                                                                   10102
[Minor, Restricts Access to/Manipulates Children, Sexual Abuse]                                                          1498
[Minor, Restricts Access to/Manipulates Children]                                                                        1065
[Forced, Minor]                                                                                                          1025
                                                                                                                        ...  
[Forced, Minor, Physical Abuse/Threat or Force, Sexual Abuse, Withholds/Destroys Important Documents]                       1
[Forced, Psychological Abuse/Threat or Force, Restricts Access to/Manipulates Children, Restricts Moveme

In [ ]:
# Use MultiLabelBinarizer to convert the target feature, y to binary format
mlb = MultiLabelBinarizer()
y_bin = mlb.fit_transform(y)

# Now `y_bin` is a binary matrix with each class represented as a column
y_df = pd.DataFrame(y_bin, columns=mlb.classes_)


In [ ]:
# Flatten the array to 1D
X_flat = X.flatten()


In [ ]:
from transformers import BertTokenizer

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the tokenization function
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')

# Apply the tokenizer to the flattened X array
X_tokenized = [tokenize_function(text) for text in X_flat]

# Print out the first tokenized sample to inspect the structure
print(X_tokenized[0])  # Inspect the first tokenized entry


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  5367,  1517,  2110,  1998,  2976,  2375,  7285,  4584,  2587,
         18079,  1012,  3958, 17357,  2368,  2012,  1037,  2811,  3034,  5958,
          2000, 12342,  1037,  2422,  2006,  2054,  1996,  3099,  2170,  1523,
          1996, 23036,  2825,  8254,  2635,  2173,  1999,  8506,  1012,  1524,
          2005,  1996,  2353,  2095,  1999,  1037,  5216,  1010, 17357,  2368,
          2006,  5958, 10116,  2254,  2004,  2529, 11626,  7073,  3204,  1999,
          8506,  1998,  4354,  1037,  2811,  3034,  2073,  2002,  1998,  2375,
          7285,  4584,  5015,  1996,  8598,  2006,  1996,  5193,  1998, 24873,
         11890,  3258,  1997,  5694,  1997,  3140,  4450,  2030,  4424, 14427,
          2008, 17357,  2368,  2056,  2003,  1523,  6230,  1999,  2035,  6109,
          5721,  1012,  1524,  1523,  2009,  2003,  7249,  1010,  1524, 17357,
          2368,  2056,  2012,  5958,  1521,  1055,  7215,  2012,  1996,  9424,
          1010,  2073,  2002,  2001,  

In [ ]:
from sklearn.model_selection import train_test_split

# Split tokenized text and labels (80% train, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X_tokenized, y_df, test_size=0.2, random_state=42)

# Further split the temporary data (X_temp, y_temp) into validation (10%) and test (10%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print sizes to confirm the splits
print(f"Training size: {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size: {len(X_test)}")


Training size: 23808
Validation size: 2976
Test size: 2976


In [ ]:
# Load Trained Model
model_path = "/content/bert_multi_label_model2.pth"

#Loading the model



In [ ]:
import torch
from transformers import BertForSequenceClassification

# Define the model path
model_path = "/content/bert_multi_label_model2.pth"  # Or the correct path in Google Drive

# Initialize the model architecture
num_labels = 15  # Adjust based on your setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification"
)

# Load the model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_path, map_location=device))

# Move the model to the correct device
model.to(device)

# Set the model to evaluation mode
model.eval()

print("✅ Model loaded successfully!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully!


In [ ]:
# Example support values from classification report (adjust as needed)
support = {
    "Alcohol/Drugs": 56,
    "Debt Bondage": 10,
    "False Promises/Deception": 5,
    "Forced": 950,
    "Lie": 121,
    "Limits/Restricts Medical Services": 6,
    "Minor": 5519,
    "Physical Abuse/Threat or Force": 91,
    "Psychological Abuse/Threat or Force": 315,
    "Restricts Access to/Manipulates Children": 740,
    "Restricts Movement/Isolates": 9,
    "Sexual Abuse": 2907,
    "Takes/Withholds Earnings/Wages": 14,
    "Use of Psychoactive Substances": 54,
    "Withholds/Destroys Important Documents": 1
}

# Calculate the inverse of the support values for class weights
total_samples = sum(support.values())
class_weights = {class_name: total_samples / count for class_name, count in support.items()}

print("Class Weights:", class_weights)



Class Weights: {'Alcohol/Drugs': 192.82142857142858, 'Debt Bondage': 1079.8, 'False Promises/Deception': 2159.6, 'Forced': 11.366315789473685, 'Lie': 89.2396694214876, 'Limits/Restricts Medical Services': 1799.6666666666667, 'Minor': 1.9565138612067403, 'Physical Abuse/Threat or Force': 118.65934065934066, 'Psychological Abuse/Threat or Force': 34.27936507936508, 'Restricts Access to/Manipulates Children': 14.591891891891892, 'Restricts Movement/Isolates': 1199.7777777777778, 'Sexual Abuse': 3.714482284141727, 'Takes/Withholds Earnings/Wages': 771.2857142857143, 'Use of Psychoactive Substances': 199.96296296296296, 'Withholds/Destroys Important Documents': 10798.0}


In [ ]:
import torch
import torch.nn as nn

# Example class weights
weights = torch.tensor([
    1.0,  # Alcohol/Drugs
    5.6,  # Debt Bondage
    11.2,  # False Promises/Deception
    0.1,  # Forced
    0.9,  # Lie
    17.6,  # Limits/Restricts Medical Services
    0.02,  # Minor
    0.1,  # Physical Abuse/Threat or Force
    0.3,  # Psychological Abuse/Threat or Force
    0.1,  # Restricts Access to/Manipulates Children
    17.3,  # Restricts Movement/Isolates
    0.02,  # Sexual Abuse
    14.8,  # Takes/Withholds Earnings/Wages
    2.3,  # Use of Psychoactive Substances
    56.0  # Withholds/Destroys Important Documents
])

# Use the class weights in BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss(weight=weights)

# Your model training loop should now use this criterion


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Convert tokenized lists into tensors
# If X_train is already a tensor, extract input_ids and attention_mask differently
if isinstance(X_train, dict):
    train_input_ids = torch.tensor(X_train["input_ids"], dtype=torch.long).to(device)
    train_attention_mask = torch.tensor(X_train["attention_mask"], dtype=torch.long).to(device)
else:
    # Convert list of dictionaries into tensors
   train_input_ids = torch.stack([x["input_ids"].squeeze(0) if isinstance(x["input_ids"], torch.Tensor) else torch.tensor(x["input_ids"], dtype=torch.long) for x in X_train]).to(device)
   train_attention_mask = torch.stack([x["attention_mask"].squeeze(0) if isinstance(x["attention_mask"], torch.Tensor) else torch.tensor(x["attention_mask"], dtype=torch.long) for x in X_train]).to(device)

# Convert labels
train_labels = torch.tensor(y_train.values, dtype=torch.float).to(device)

val_input_ids = torch.stack([x["input_ids"].squeeze(0) if isinstance(x["input_ids"], torch.Tensor) else torch.tensor(x["input_ids"], dtype=torch.long) for x in X_val]).to(device)
val_attention_mask = torch.stack([x["attention_mask"].squeeze(0) if isinstance(x["attention_mask"], torch.Tensor) else torch.tensor(x["attention_mask"], dtype=torch.long) for x in X_val]).to(device)

val_labels = torch.tensor(y_val.values, dtype=torch.float).to(device)

test_input_ids = torch.stack([x["input_ids"] for x in X_test]).to(device)
test_attention_mask = torch.stack([x["attention_mask"] for x in X_test]).to(device)


test_labels = torch.tensor(y_test.values, dtype=torch.float).to(device)

# Create TensorDatasets
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

# Create DataLoaders
batch_size = 16  # Adjust batch size as needed
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("✅ Data is correctly formatted and DataLoaders are ready!")


✅ Data is correctly formatted and DataLoaders are ready!


In [ ]:
from transformers import BertForSequenceClassification
from transformers import AdamW
import torch
import torch.nn as nn

# Define the model path
model_path = "/content/bert_multi_label_model2.pth"  # Or the correct path in Google Drive

# Initialize the model architecture
num_labels = 15  # Adjust based on your setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification"
)

# Load the model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_path, map_location=device))

# Move the model to the correct device
model.to(device)

# Set the model to evaluation mode (optional for fine-tuning)
model.eval()

# Define class weights based on the classification report
weights = torch.tensor([
    1.0,  # Alcohol/Drugs
    5.6,  # Debt Bondage
    11.2,  # False Promises/Deception
    0.1,  # Forced
    0.9,  # Lie
    17.6,  # Limits/Restricts Medical Services
    0.02,  # Minor
    0.1,  # Physical Abuse/Threat or Force
    0.3,  # Psychological Abuse/Threat or Force
    0.1,  # Restricts Access to/Manipulates Children
    17.3,  # Restricts Movement/Isolates
    0.02,  # Sexual Abuse
    14.8,  # Takes/Withholds Earnings/Wages
    2.3,  # Use of Psychoactive Substances
    56.0  # Withholds/Destroys Important Documents
]).to(device)

# Define the loss function with class weights
criterion = nn.BCEWithLogitsLoss(weight=weights).to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 3  # Define the number of epochs for fine-tuning

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch in train_dataloader:  # Replace with your actual DataLoader
        optimizer.zero_grad()  # Clear the gradients

        # Get input data from the batch
        input_ids, attention_mask, labels = batch

         # Move tensors to the correct device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Get the logits from the model output

        # Compute the loss with the class weights
        loss = criterion(logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Optionally, evaluate after each epoch to monitor the validation performance
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

print("✅ Model fine-tuned successfully!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.02150985598564148
Epoch 2/3, Loss: 0.018855106085538864
Epoch 3/3, Loss: 0.008407040499150753
✅ Model fine-tuned successfully!


In [ ]:
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Evaluation loop
model.eval()  # Set the model to evaluation mode

all_preds = []
all_labels = []

with torch.no_grad():  # Disable gradient calculation for inference
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.squeeze(1).to(device)  # Remove the extra dimension
        attention_mask = attention_mask.squeeze(1).to(device)  # Remove the extra dimension
        labels = labels.to(device)

        # Forward pass through the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Extract logits

        # Apply sigmoid to get probabilities
        probs = torch.sigmoid(logits)

        # Convert probabilities to binary predictions (threshold=0.5)
        preds = (probs > 0.5).float()

        # Store predictions and labels for evaluation
        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

# Concatenate all predictions and labels
all_preds = torch.cat(all_preds, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()

# Calculate evaluation metrics
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

# Print evaluation results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")



Accuracy: 0.8182
Precision: 0.9340
Recall: 0.9295
F1-score: 0.9284


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import classification_report, hamming_loss, jaccard_score


In [ ]:
from sklearn.metrics import classification_report, hamming_loss, jaccard_score

# Calculate classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds))

# Print evaluation results
print(f"Accuracy: {accuracy:.4f}")

print(f"Precision: {precision:.4f}")

print(f"Recall: {recall:.4f}")

print(f"F1-score: {f1:.4f}")

# Calculate Hamming Loss
hamming = hamming_loss(all_labels, all_preds)
print(f"Hamming Loss: {hamming:.4f}")

# Calculate Jaccard Score (this is per label, or 'samples' for overall average)
jaccard = jaccard_score(all_labels, all_preds, average='samples')
print(f"Jaccard Score: {jaccard:.4f}")


Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.64      0.70        25
           1       0.67      0.33      0.44         6
           2       0.00      0.00      0.00         2
           3       0.94      0.92      0.93       484
           4       0.73      0.14      0.24        57
           5       0.00      0.00      0.00         2
           6       0.97      1.00      0.98      2746
           7       0.20      0.02      0.04        44
           8       0.79      0.67      0.73       147
           9       0.79      0.73      0.76       363
          10       0.50      0.20      0.29         5
          11       0.95      0.96      0.96      1459
          12       0.50      0.11      0.18         9
          13       0.78      0.78      0.78        23
          14       0.33      1.00      0.50         1

   micro avg       0.94      0.93      0.94      5373
   macro avg       0.59      0.50      0.50      5373
wei

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Saving the entire model with the weight and architecture**

In [ ]:
# Define the new model path with a different name
new_model_path = "/content/bert_multi_label_model_v2.pth"  # Change the filename as needed

# Save the entire model with the new name
torch.save(model, new_model_path)
print(f"✅ Entire model saved successfully at {new_model_path}")


✅ Entire model saved successfully at /content/bert_multi_label_model_v2.pth


**Saving the model only**

In [ ]:
# Define the model path
model_path = "bert_model22.pth"  # Change the name and path as needed

# Save the model's state_dict (weights and architecture)
torch.save(model.state_dict(), model_path)

print(f"✅ Model saved to {model_path}")


✅ Model saved to bert_model22.pth


In [ ]:


# Assuming you have already computed the classification report
report = classification_report(all_labels, all_preds, output_dict=True)

# Convert the classification report to a pandas DataFrame
report_df_bert2 = pd.DataFrame(report).transpose()

# Save the DataFrame to a CSV file
report_df_bert2.to_csv("bert2_classification_report.csv", index=True)

print("✅ Classification report saved as CSV")



✅ Classification report saved as CSV


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
